<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Start-to-Finish Example: `GiRaFFE_NRPy` 1D tests

### Authors: Patrick Nelson & Terrence Pierre Jacques

### Adapted from [Start-to-Finish Example: Head-On Black Hole Collision](../Tutorial-Start_to_Finish-BSSNCurvilinear-Two_BHs_Collide.ipynb)

## This module compiles and runs code tests for all 1D initial data options available in GiRaFFE-NRPy+, evolving one-dimensional GRFFE waves.

### NRPy+ Source Code for this module: 

* Main python module for all 1D initial data: [GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_1D_tests.py](../../edit/in_progress/GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_1D_tests.py) __Options:__
    1. [Fast Wave](Tutorial-GiRaFFEfood_NRPy_1D_tests-fast_wave.ipynb)
    1. [Alfven Wave](Tutorial-GiRaFFEfood_NRPy_1D_alfven_wave.ipynb)
    1. [Degenerate Alfven Wave](Tutorial-GiRaFFEfood_NRPy_1D_tests-degen_Alfven_wave.ipynb)
    1. [Three Alfven Waves](Tutorial-GiRaFFEfood_NRPy_1D_tests-three_waves.ipynb)
    1. [FFE Breakdown](Tutorial-GiRaFFEfood_NRPy_1D_tests-FFE_breakdown.ipynb)
* [GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_Afield_flux.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_Afield_flux.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy_staggered-Afield_flux.ipynb) Generates the expressions to find the flux term of the induction equation.
* [GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_A2B.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_A2B.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy_staggered-A2B.ipynb) Generates the driver to compute the magnetic field from the vector potential/
* [GiRaFFE_NRPy/GiRaFFE_NRPy_BCs.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_BCs.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy-BCs.ipynb) Generates the code to apply boundary conditions to the vector potential, scalar potential, and three-velocity.
* [GiRaFFE_NRPy/GiRaFFE_NRPy_C2P_P2C.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_C2P_P2C.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy-C2P_P2C.ipynb) Generates the conservative-to-primitive and primitive-to-conservative solvers.
* [GiRaFFE_NRPy/GiRaFFE_NRPy_Metric_Face_Values.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_Metric_Face_Values.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy-Metric_Face_Values.ipynb) Generates code to interpolate metric gridfunctions to cell faces.
* [GiRaFFE_NRPy/GiRaFFE_NRPy_PPM.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_PPM.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy-PPM.ipynb) Genearates code to reconstruct primitive variables on cell faces.
* [GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_Source_Terms.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_Source_Terms.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy_staggered-Source_Terms.ipynb) Generates the expressions to find the flux term of the Poynting flux evolution equation.
* [GiRaFFE_NRPy/Stilde_flux.py](../../edit/in_progress/GiRaFFE_NRPy/Stilde_flux.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy-Stilde_flux.ipynb) Generates the expressions to find the flux term of the Poynting flux evolution equation.
* [../GRFFE/equations.py](../../edit/GRFFE/equations.py) [\[**tutorial**\]](../Tutorial-GRFFE_Equations-Cartesian.ipynb) Generates code necessary to compute the source terms.
* [../GRHD/equations.py](../../edit/GRHD/equations.py) [\[**tutorial**\]](../Tutorial-GRHD_Equations-Cartesian.ipynb) Generates code necessary to compute the source terms.

Here we use NRPy+ to generate the C source code necessary to set up initial data for an Alfv&eacute;n wave (see [the original GiRaFFE paper](https://arxiv.org/pdf/1704.00599.pdf)). Then we use it to generate the RHS expressions for [Method of Lines](https://reference.wolfram.com/language/tutorial/NDSolveMethodOfLines.html) time integration based on the [explicit Runge-Kutta fourth-order scheme](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) (RK4).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Set core NRPy+ parameters for numerical grids
1. [Step 2](#grffe): Output C code for GRFFE evolution
    1. [Step 2.a](#mol): Output macros for Method of Lines timestepping
1. [Step 3](#gf_id): Import `GiRaFFEfood_NRPy` initial data modules
1. [Step 4](#cparams): Output C codes needed for declaring and setting Cparameters; also set `free_parameters.h`
1. [Step 5](#mainc): `GiRaFFE_NRPy_standalone.c`: The Main C Code
1. [Step 6](#compileexec): Compile and execute C codes
1. [Step 7](#plots): Data Visualization
1. [Step 8](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='setup'></a>

# Step 1: Set up core functions and parameters for solving  GRFFE equations \[Back to [top](#toc)\]
$$\label{setup}$$


In [1]:
import shutil, os, sys           # Standard Python modules for multiplatform OS-level functions
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
GRMHD_dir_path = os.path.join("../GRMHD_formulation/")
sys.path.append(GRMHD_dir_path)
import GRMHD_equations_new_version as GRMHD    # NRPy+: Generate general relativistic magnetohydrodynamics equations

src_dir_path = os.path.join("../source_terms/")
sys.path.append(src_dir_path)
import IGM_All_Source_Terms as ST   

fluxes_dir_path = os.path.join("../fluxes/")
sys.path.append(fluxes_dir_path)
import IGM_All_fluxes as FL   
import IGM_Characteristic_Speeds as chsp 


nrpy_dir_path = os.path.join("../nrpy/nrpytutorial/")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
# Step P1: Import needed NRPy+ core modules:
from outputC import nrpyAbs, lhrh, outputC, outCfunction, add_to_Cfunction_dict, lhrh # NRPy+: Core C code output module
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import loop as lp                # NRPy+: Generate C code loops
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

import CurviBoundaryConditions.CurviBoundaryConditions_new_way as CBC

# Step P2: Create C code output directory:
Ccodesrootdir = os.path.join("IGM_Flux_SRC_UnitTest_standalone_Ccodes_fixes_flux/")
# First remove C code output directory if it exists
# Courtesy https://stackoverflow.com/questions/303200/how-do-i-remove-delete-a-folder-that-is-not-empty
# !rm -r ScalarWaveCurvilinear_Playground_Ccodes
shutil.rmtree(Ccodesrootdir, ignore_errors=True)
# # Then create a fresh directory
cmd.mkdir(Ccodesrootdir)

# Step P3: Create executable output directory:
outdir = os.path.join(Ccodesrootdir, "output")
cmd.mkdir(outdir)

# Step P4: Enable/disable SIMD. If enabled, code should run ~2x faster on most CPUs.
enable_SIMD = False

# Step P5: Enable reference metric precomputation.
enable_rfm_precompute = False
par.set_parval_from_str("reference_metric::rfm_precompute_to_Cfunctions_and_NRPy_basic_defines", "True")

if enable_SIMD and not enable_rfm_precompute:
    print("ERROR: SIMD does not currently handle transcendental functions,\n")
    print("       like those found in rfmstruct (rfm_precompute).\n")
    print("       Therefore, enable_SIMD==True and enable_rfm_precompute==False\n")
    print("       is not supported.\n")
    sys.exit(1)

# Step P6: Enable "FD functions". In other words, all finite-difference stencils
#         will be output as inlined static functions. This is essential for
#         compiling highly complex FD kernels with using certain versions of GCC;
#         GCC 10-ish will choke on BSSN FD kernels at high FD order, sometimes
#         taking *hours* to compile. Unaffected GCC versions compile these kernels
#         in seconds. FD functions do not slow the code performance, but do add
#         another header file to the C source tree.
# With gcc 7.5.0, enable_FD_functions=True decreases performance by 10%
enable_FD_functions = False

# Step 1: Set some core parameters, including CoordSystem, boundary condition,
#                                             MoL, timestepping algorithm, FD order,
#                                             floating point precision, and CFL factor:

# Step 1.a: Set the coordinate system for the numerical grid
# Choices are: Spherical, SinhSpherical, SinhSphericalv2, Cylindrical, SinhCylindrical,
#              SymTP, SinhSymTP
CoordSystem     = "Cartesian"
par.set_parval_from_str("reference_metric::CoordSystem", CoordSystem)
rfm.reference_metric()

# Step 1.b: Set outer boundary condition
# Current choices are extrapolation (quadratic polynomial extrapolation) or radiation
# OuterBoundaryCondition = "radiation"
OuterBoundaryCondition = "extrapolation"

# Step 1.c: Set defaults for Coordinate system parameters.
#           These are perhaps the most commonly adjusted parameters,
#           so we enable modifications at this high level.

# domain_size sets the default value for:
#   * Spherical's params.RMAX
#   * SinhSpherical*'s params.AMAX
#   * Cartesians*'s -params.{x,y,z}min & .{x,y,z}max
#   * Cylindrical's -params.ZMIN & .{Z,RHO}MAX
#   * SinhCylindrical's params.AMPL{RHO,Z}
#   * *SymTP's params.AMAX
# domain_size     = 10.0 # Needed for all coordinate systems.
domain_size     = 1.6

# sinh_width sets the default value for:
#   * SinhSpherical's params.SINHW
#   * SinhCylindrical's params.SINHW{RHO,Z}
#   * SinhSymTP's params.SINHWAA
sinh_width      = 0.4 # If Sinh* coordinates chosen

# sinhv2_const_dr sets the default value for:
#   * SinhSphericalv2's params.const_dr
#   * SinhCylindricalv2's params.const_d{rho,z}
sinhv2_const_dr = 0.05# If Sinh*v2 coordinates chosen

# SymTP_bScale sets the default value for:
#   * SinhSymTP's params.bScale
SymTP_bScale    = 1.0 # If SymTP chosen

# Step 1.d: Set the order of spatial and temporal derivatives;
#           the core data type, and the CFL factor.
# RK_method choices include: Euler, "RK2 Heun", "RK2 MP", "RK2 Ralston", RK3, "RK3 Heun", "RK3 Ralston",
#              SSPRK3, RK4, DP5, DP5alt, CK5, DP6, L6, DP8
RK_method = "RK4"
FD_order  = 2        # Finite difference order: even numbers only, starting with 2. 12 is generally unstable
REAL      = "double" # Best to use double here.
CFL_FACTOR= 0.5


# Step 5: Set the finite differencing order to FD_order (set above).
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", FD_order)

thismodule = __name__
TINYDOUBLE = par.Cparameters("REAL", thismodule, "TINYDOUBLE", 1e-100)


<a id='grffe'></a>

# Step 2: Output C code for GRFFE evolution \[Back to [top](#toc)\]
$$\label{grffe}$$

We will first write the C codes needed for GRFFE evolution. We have already written a module to generate all these codes and call the functions in the appropriate order, so we will import that here. We will take the slightly unusual step of doing this before we generate the initial data functions because the main driver module will register all the gridfunctions we need. It will also generate functions that, in addition to their normal spot in the MoL timestepping, will need to be called during the initial data step to make sure all the variables are appropriately filled in. 

All of this is handled with a single call to `GiRaFFE_NRPy_Main_Driver_generate_all()`, which will register gridfunctions, write all the C code kernels, and write the C code functions to call those.

In [2]:
alpha = gri.register_gridfunctions("AUXEVOL","alpha")
gammaDD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","gammaDD","sym01",DIM=3)
betaU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","betaU", DIM=3)

alpha_dD = ixp.declarerank1("alpha_dD", DIM=3)
gammaDD_dD = ixp.declarerank3("gammaDD_dD","sym01",DIM=3)
betaU_dD = ixp.declarerank2("betaU_dD", "nosym", DIM=3)

tmp1D = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","tmp1D",DIM=3)
tmp2DD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","tmp2DD","nosym", DIM=3)
tmp3DDD = ixp.register_gridfunctions_for_single_rank3("AUXEVOL","tmp3DDD","sym01", DIM=3)

expr_list = []
for i in range(3):
    expr_list.append(lhrh(lhs=gri.gfaccess("out_gfs",fr"tmp1D{i}"),rhs=alpha_dD[i]))
    for j in range(3):
        expr_list.append(lhrh(lhs=gri.gfaccess("out_gfs",fr"tmp2DD{i}{j}"),rhs=betaU_dD[i][j]))

for j in range(3):
    i = 0
    for k in range(3):
        expr_list.append(lhrh(lhs=gri.gfaccess("out_gfs",fr"tmp3DDD{i}{j}{k}"),rhs=gammaDD_dD[i][j][k]))       

for j in range(1,3):
    i = 1
    for k in range(3):
        expr_list.append(lhrh(lhs=gri.gfaccess("out_gfs",fr"tmp3DDD{i}{j}{k}"),rhs=gammaDD_dD[i][j][k]))      

i = 2
j = 2
for k in range(3):
    expr_list.append(lhrh(lhs=gri.gfaccess("out_gfs",fr"tmp3DDD{i}{j}{k}"),rhs=gammaDD_dD[i][j][k]))

outCparams = "outCverbose=False,CSE_sorting=canonical,CSE_enable=True"
desc = "Calculate metric derivatives"
includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
name = "calculate_metric_derivatives"
body = fin.FD_outputC("returnstring",
                      expr_list,
                      params=outCparams)
body = body.replace("TMP1D", "ALPHA_DD")
body = body.replace("TMP2DD", "BETAU_DD")
body = body.replace("TMP3DDD", "GAMMADD_DD")

c_type = "void"
params   ="const paramstruct *restrict params, "
params += "REAL *restrict xx[3], "
params += "REAL *restrict auxevol_gfs"
loopopts = "InteriorPoints"
loopopts += ",Read_xxs"
add_to_Cfunction_dict(
    includes=includes,
    desc=desc,
    c_type=c_type, name=name, params=params,
    body=body,
    rel_path_to_Cparams=os.path.join("."), loopopts = loopopts)

gri.glb_gridfcs_list = []

In [3]:
formalism="ADM"

alpha_face = gri.register_gridfunctions("AUXEVOL","alpha_face")
# cf_face = gri.register_gridfunctions("AUXEVOL","cf_face")
gamma_faceDD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","gamma_faceDD","sym01",DIM=3)
beta_faceU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","beta_faceU", DIM=3)

alpha = gri.register_gridfunctions("AUXEVOL","alpha")
alpha_dD = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","alpha_dD", DIM=3)

# cf = gri.register_gridfunctions("AUXEVOL","cf")

gammaDD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","gammaDD","sym01",DIM=3)
gammaDD_dD = ixp.register_gridfunctions_for_single_rank3("AUXEVOL","gammaDD_dD","sym01",DIM=3)

KDD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","KDD","sym01",DIM=3)
# KDD_dD = ixp.register_gridfunctions_for_single_rank3("AUXEVOL","KDD_dD","sym01",DIM=3)

betaU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","betaU", DIM=3)
betaU_dD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","betaU_dD", "nosym", DIM=3)

# We'll need some more gridfunctions, now, to represent the reconstructions of BU and ValenciavU
# on the right and left faces
u4_rU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","u4_rU",DIM=4)
V_rU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","V_rU",DIM=3)
B_rU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","B_rU",DIM=3)

u4_lU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","u4_lU",DIM=4)
V_lU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","V_lU",DIM=3)
B_lU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","B_lU",DIM=3)

u4U = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","u4U",DIM=4)
VU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","VU",DIM=3)
BU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","BU",DIM=3)

rho_star = gri.register_gridfunctions("EVOL","rho_star")
tau_tilde = gri.register_gridfunctions("EVOL","tau_tilde")
StildeD = ixp.register_gridfunctions_for_single_rank1("EVOL","StildeD")

HLLE_flux_StildeD = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","HLLE_flux_StildeD")
HLLE_flux_rho_star = gri.register_gridfunctions("AUXEVOL","HLLE_flux_rho_star")
HLLE_flux_tau_tilde = gri.register_gridfunctions("AUXEVOL","HLLE_flux_tau_tilde")

h = gri.register_gridfunctions("AUXEVOL", "h")
h_r = gri.register_gridfunctions("AUXEVOL", "h_r")
h_l = gri.register_gridfunctions("AUXEVOL", "h_l")

P = gri.register_gridfunctions("AUXEVOL", "P")
P_r = gri.register_gridfunctions("AUXEVOL", "P_r")
P_l = gri.register_gridfunctions("AUXEVOL", "P_l")

rho_b = gri.register_gridfunctions("AUXEVOL", "rhob")
rho_b_r = gri.register_gridfunctions("AUXEVOL", "rhob_r")
rho_b_l = gri.register_gridfunctions("AUXEVOL", "rhob_l")

Gamma_th_r = gri.register_gridfunctions("AUXEVOL", "Gamma_th_r")
Gamma_th_l = gri.register_gridfunctions("AUXEVOL", "Gamma_th_l")

epsilon_th_r = gri.register_gridfunctions("AUXEVOL", "epsilon_th_r")
epsilon_th_l = gri.register_gridfunctions("AUXEVOL", "epsilon_th_l")

dPcold_drhob_r = gri.register_gridfunctions("AUXEVOL", "dPcold_drhob_r")
dPcold_drhob_l = gri.register_gridfunctions("AUXEVOL", "dPcold_drhob_l")


In [4]:
ST.add_to_Cfunction_dict__Stilde_SourceTerms(formalism=formalism)
chsp.add_to_Cfunction_dict__GRMHD_characteristic_speeds(formalism=formalism)
FL.add_to_Cfunction_dict__GRMHD_fluxes(formalism=formalism)

<a id='mol'></a>

## Step 2.a: Output macros for Method of Lines timestepping \[Back to [top](#toc)\]
$$\label{mol}$$

Now, we generate the code to implement the method of lines using the fourth-order Runge-Kutta algorithm.

<a id='cparams'></a>

# Step 4: Output C codes needed for declaring and setting Cparameters; also set `free_parameters.h` \[Back to [top](#toc)\]
$$\label{cparams}$$

Based on declared NRPy+ Cparameters, first we generate `declare_Cparameters_struct.h`, `set_Cparameters_default.h`, and `set_Cparameters[-SIMD].h`.

Then we output `free_parameters.h`, which sets initial data parameters, as well as grid domain & reference metric parameters, applying `domain_size` and `sinh_width`/`SymTP_bScale` (if applicable) as set above

In [5]:
# Step 1.c.i: Set free_parameters.h
outstr = r"""
// Free parameters related to physical system:
"""

# Append to $Ccodesrootdir/free_parameters.h reference metric parameters based on generic
#    domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale,
#    parameters set above.
outstr += rfm.out_default_free_parameters_for_rfm("returnstring",
                                                  domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale)
with open(os.path.join(Ccodesrootdir,"free_parameters.h"),"w") as file:
    file.write(outstr.replace("params.", "griddata.params."))

In [6]:
GRMHD.set_up_base_vars(formalism=formalism)

GRMHD.compute_vU_from_u4U__no_speed_limit(GRMHD.u4U)   

GRMHD.compute_sqrtgammaDET(GRMHD.gammaDD)
GRMHD.compute_smallb4U(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.u4U, GRMHD.BU, GRMHD.sqrt4pi)
GRMHD.compute_smallbsquared(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.smallb4U)

# First compute stress-energy tensor T4UU and T4UD:
GRMHD.compute_T4UU(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.rho_b,GRMHD.P,GRMHD.h,GRMHD.u4U, GRMHD.smallb4U, GRMHD.smallbsquared)
GRMHD.compute_T4UD(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.T4UU)

# Compute conservative variables in terms of primitive variables
GRMHD.compute_rho_star(GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.rho_b,GRMHD.u4U)
GRMHD.compute_tau_tilde(GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.T4UU,GRMHD.rho_star)
GRMHD.compute_S_tildeD(GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.T4UD)

# Next compute fluxes of conservative variables
GRMHD.compute_rho_star_fluxU(GRMHD.VU,GRMHD.rho_star)
GRMHD.compute_tau_tilde_fluxU(GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.VU, GRMHD.T4UU, GRMHD.rho_star)
GRMHD.compute_S_tilde_fluxUD (GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.T4UD)

# Then declare derivatives & compute g4DD_zerotimederiv_dD
GRMHD.compute_g4DD_zerotimederiv_dD(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.gammaDD_dD,GRMHD.betaU_dD,GRMHD.alpha_dD)

# Then compute source terms on tau_tilde and S_tilde equations
GRMHD.compute_tau_tilde_source_term(GRMHD.KDD,GRMHD.betaU,GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.alpha_dD, GRMHD.T4UU)
GRMHD.compute_S_tilde_source_termD(GRMHD.alpha,GRMHD.sqrtgammaDET,GRMHD.g4DD_zerotimederiv_dD, GRMHD.T4UU)


tau_tilde_source_term_free_symbols = GRMHD.tau_tilde_source_term.free_symbols
S_tilde_source_termD0_free_symbols = GRMHD.S_tilde_source_termD[0].free_symbols
S_tilde_source_termD1_free_symbols = GRMHD.S_tilde_source_termD[1].free_symbols
S_tilde_source_termD2_free_symbols = GRMHD.S_tilde_source_termD[2].free_symbols

all_free_sysmbols = tau_tilde_source_term_free_symbols.union(\
                             S_tilde_source_termD0_free_symbols, 
                             S_tilde_source_termD1_free_symbols,
                             S_tilde_source_termD2_free_symbols)       

In [7]:
local_prims_vectors = ["u4U0", "u4U1", "u4U2", "u4U3",
               "BU0", "BU1", "BU2"]

local_prims_scalars = ["P", "h", "rhob", "Gamma_th", "epsilon_th", "dPcold_drhob"]

local_metric_vars = ["alpha", "betaU0", "betaU1", "betaU2",
                    "gammaDD01", "gammaDD02", "gammaDD12", 
                    "gammaDD00", "gammaDD11", "gammaDD22",
                    "KDD01", "KDD02", "KDD12", 
                    "KDD00", "KDD11", "KDD22"]

loop_body = r"""
"""

for p_v in local_prims_vectors:
    #capitalize str
    P_V = p_v.upper()
    loop_body += fr"""    
    reconstructed_prims->{p_v} = auxevol_gfs[IDX4ptS({P_V}GF, ijk)];
    reconstructed_prims_r->{p_v} = auxevol_gfs[IDX4ptS({p_v.replace("U", "_RU").upper()}GF, ijk)];
    reconstructed_prims_l->{p_v} = auxevol_gfs[IDX4ptS({p_v.replace("U", "_LU").upper()}GF, ijk)];
"""
    
for p_v in local_prims_scalars:
    #capitalize str
    P_V = p_v.upper()
    loop_body += fr"""
    reconstructed_prims_r->{p_v} = auxevol_gfs[IDX4ptS({P_V}_RGF, ijk)];
    reconstructed_prims_l->{p_v} = auxevol_gfs[IDX4ptS({P_V}_LGF, ijk)];
"""
    if p_v!="Gamma_th" and p_v!="epsilon_th" and p_v!="dPcold_drhob":
        loop_body += fr"""
    reconstructed_prims->{p_v} = auxevol_gfs[IDX4ptS({P_V}GF, ijk)];
"""
        
for m_v in local_metric_vars:
    m_v_face = m_v.replace("alpha", "alpha_face").replace("U", "_faceU").replace("DD", "_faceDD")
    M_V = m_v.upper()
    M_V_face = m_v_face.upper()
    loop_body += fr"""    
    metric_quantities->{m_v} = auxevol_gfs[IDX4ptS({M_V}GF, ijk)];
"""
    if "KDD" not in m_v:
        loop_body += fr""" 
    metric_face_quantities->{m_v_face} = auxevol_gfs[IDX4ptS({M_V_face}GF, ijk)];
"""
    
for var in all_free_sysmbols:
    if "_dD" in str(var):
        loop_body += fr"""
    metric_quantities_derivatives->{str(var)} = auxevol_gfs[IDX4ptS({str(var).upper()}GF, ijk)];
"""

c_type = "void"
params   = "const int ijk, const paramstruct *params, const double *restrict auxevol_gfs, "
params  += "reconstructed_prims_struct *restrict reconstructed_prims, "
params  += "reconstructed_prims_struct *restrict reconstructed_prims_r, "
params  += "reconstructed_prims_struct *restrict reconstructed_prims_l, " 
params  += "metric_quantities_struct *restrict metric_quantities, "
params  += "metric_face_quantities_struct *restrict metric_face_quantities, "
params  += "metric_quantities_derivatives_struct *restrict metric_quantities_derivatives"

desc = "Initialize structs used for flux and source calculations"
name = "initialize_structs"
includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]

add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        name=name,
        params=params,
        body=loop_body, 
        rel_path_to_Cparams=os.path.join("."))

<a id='bc_functs'></a>

# Step 4: Set up boundary condition functions for chosen singular, curvilinear coordinate system \[Back to [top](#toc)\]
$$\label{bc_functs}$$

Next apply singular, curvilinear coordinate boundary conditions [as documented in the corresponding NRPy+ tutorial notebook](Tutorial-Start_to_Finish-Curvilinear_BCs.ipynb)

...But, for the moment, we're actually just using this because it writes the file `gridfunction_defines.h`.

In [8]:
# First download the original IllinoisGRMHD source code
import urllib
original_file_url  = ["https://bitbucket.org/zach_etienne/wvuthorns/raw/0cb4d23a59e511b9a3607178d9ca373c582e70d4/IllinoisGRMHD/src/add_fluxes_and_source_terms_to_hydro_rhss.C",
                      "https://bitbucket.org/zach_etienne/wvuthorns/raw/0cb4d23a59e511b9a3607178d9ca373c582e70d4/IllinoisGRMHD/src/compute_tau_rhs_extrinsic_curvature_terms_and_TUPmunu.C",
                      "https://bitbucket.org/zach_etienne/wvuthorns/raw/0cb4d23a59e511b9a3607178d9ca373c582e70d4/IllinoisGRMHD/src/inlined_functions.C",
                      "https://bitbucket.org/zach_etienne/wvuthorns/raw/0cb4d23a59e511b9a3607178d9ca373c582e70d4/IllinoisGRMHD/src/mhdflux.C",
                      "https://bitbucket.org/zach_etienne/wvuthorns/raw/0cb4d23a59e511b9a3607178d9ca373c582e70d4/IllinoisGRMHD/src/driver_evaluate_MHD_rhs.C",
                      "https://bitbucket.org/zach_etienne/wvuthorns/raw/0cb4d23a59e511b9a3607178d9ca373c582e70d4/IllinoisGRMHD/src/driver_evaluate_MHD_rhs.h",
                      "https://bitbucket.org/zach_etienne/wvuthorns/raw/0cb4d23a59e511b9a3607178d9ca373c582e70d4/IllinoisGRMHD/src/reconstruct_set_of_prims_PPM.C",
                      "https://bitbucket.org/zach_etienne/wvuthorns/raw/0cb4d23a59e511b9a3607178d9ca373c582e70d4/IllinoisGRMHD/src/loop_defines_reconstruction.h"
                     ]

original_file_name = ["add_fluxes_and_source_terms_to_hydro_rhss.C",
                      "compute_tau_rhs_extrinsic_curvature_terms_and_TUPmunu.C",
                      "inlined_functions.C",
                      "mhdflux.C",
                      "driver_evaluate_MHD_rhs.C",
                      "driver_evaluate_MHD_rhs.h",
                      "reconstruct_set_of_prims_PPM.C",
                      "loop_defines_reconstruction.h"
                     ]

for i in range(len(original_file_url)):
    original_file_path = os.path.join(Ccodesrootdir,original_file_name[i])

    # Then download the original IllinoisGRMHD source code
    # We try it here in a couple of ways in an attempt to keep
    # the code more portable
    try:
        original_file_code = urllib.request.urlopen(original_file_url[i]).read().decode('utf-8')
    except:
        original_file_code = urllib.urlopen(original_file_url[i]).read().decode('utf-8')

    # Write down the file the original IllinoisGRMHD source code
    with open(original_file_path,"w") as file:
        original_file_code = original_file_code.replace("&eos", "*restrict eos")
        original_file_code = original_file_code.replace("eos.", "eos->")
        original_file_code = original_file_code.replace("&stats", "*restrict stats")
        original_file_code = original_file_code.replace("stats.", "stats->")
        original_file_code = original_file_code.replace(",stats,", ",&stats,")
        original_file_code = original_file_code.replace("output_stats stats; stats->", 
                                                        "output_stats stats; stats.")
#         original_file_code = original_file_code.replace(",eos,", ",&eos,")        
        original_file_code = original_file_code.replace("&P_cold", "P_cold")
        original_file_code = original_file_code.replace("&gamma_cold", "gamma_cold")
        original_file_code = original_file_code.replace("&Ul", "Ul")
        original_file_code = original_file_code.replace("&Ur", "Ur")
        original_file_code = original_file_code.replace("&c", "c")
        original_file_code = original_file_code.replace("&v", "v")
        original_file_code = original_file_code.replace("&u", "u")
        original_file_code = original_file_code.replace("&P", "P")
        original_file_code = original_file_code.replace("&dPcold_drho", "dPcold_drho")
        original_file_code = original_file_code.replace("&eps_th", "eps_th")
        original_file_code = original_file_code.replace("&eps_cold", "eps_cold")
        original_file_code = original_file_code.replace("&h", "h")
        original_file_code = original_file_code.replace("&rho_star", "rho_star")
        original_file_code = original_file_code.replace("&tau_flux", "tau_flux")
        original_file_code = original_file_code.replace("&st_", "st_")
#         original_file_code = original_file_code.replace("CCTK_GFINDEX3D(cctkGH,", "IDX3S(")
#         original_file_code = original_file_code.replace("CCTK_GFINDEX3D(cctkGH ,", "IDX3S(")
#         original_file_code = original_file_code.replace("const cGH *cctkGH,", "")
#         original_file_code = original_file_code.replace("const cGH *cctkGH ,", "")
#         original_file_code = original_file_code.replace("cctkGH,", "")
        original_file_code = original_file_code.replace("struct output_stats stats;", "output_stats stats;")
        original_file_code = original_file_code.replace("DECLARE_CCTK", "// DECLARE_CCTK")
        original_file_code = original_file_code.replace("if(std::isnan(alpha_u0*ONE_OVER_LAPSE)", "// if(std::isnan(alpha_u0*ONE_OVER_LAPSE)")
        original_file_code = original_file_code.replace("CCTK_VError(VERR_DEF_PARAMS,", 
                                                        "printf(")
#         original_file_code = original_file_code.replace("static inline", "inline")
#         original_file_code = original_file_code.replace("static void", "void")
        file.write(original_file_code)

<a id='mainc'></a>

# Step 5: `GiRaFFE_NRPy_standalone.c`: The Main C Code \[Back to [top](#toc)\]
$$\label{mainc}$$

In [9]:
def add_to_Cfunction_dict_main__IGM_Playground():
    includes = [  "NRPy_basic_defines.h",
                  "NRPy_function_prototypes.h",
                  "inlined_functions.C",
                  "reconstruct_set_of_prims_PPM.C",
                  "loop_defines_reconstruction.h",
                  "add_fluxes_and_source_terms_to_hydro_rhss.C",
                  "compute_tau_rhs_extrinsic_curvature_terms_and_TUPmunu.C",
                  "mhdflux.C"
                  ]
    
    
    desc = """// main() function:
// Step 0: Read command-line input, set up grid structure, allocate memory for gridfunctions, set up coordinates
// Step 1: Write test data to gridfunctions
// Step 2: Overwrite all data in ghost zones with NaNs
// Step 3: Apply curvilinear boundary conditions
// Step 4: Print gridfunction data after curvilinear boundary conditions have been applied
// Step 5: Free all allocated memory
"""
    c_type = "int"
    name = "main"
    params = "int argc, const char *argv[]"
    body = r"""  griddata_struct griddata;
  set_Cparameters_to_default(&griddata.params);

  // Step 0a: Read command-line input, error out if nonconformant
  /* if(argc != 5 || atoi(argv[1]) < NGHOSTS || atoi(argv[2]) < NGHOSTS || atoi(argv[3]) < NGHOSTS) {
    printf("Error: Expected one command-line argument: ./ScalarWaveCurvilinear_Playground Nx0 Nx1 Nx2,\n");
    printf("where Nx[0,1,2] is the number of grid points in the 0, 1, and 2 directions.\n");
    printf("Nx[] MUST BE larger than NGHOSTS (= %d)\n",NGHOSTS);
    exit(1);
  } */
  
  // Step 0b: Set up numerical grid structure, first in space...
  const int Nxx[3] = { atoi(argv[1]), atoi(argv[2]), atoi(argv[3]) };
  if(Nxx[0]%2 != 0 || Nxx[1]%2 != 0 || Nxx[2]%2 != 0) {
    printf("Error: Cannot guarantee a proper cell-centered grid if number of grid cells not set to even number.\n");
    printf("       For example, in case of angular directions, proper symmetry zones will not exist.\n");
    exit(1);
  }

  // Step 0c: Set free parameters, overwriting Cparameters defaults
  //          by hand or with command-line input, as desired.
#include "free_parameters.h"

  {
    int EigenCoord = 0;
    // Step 0d.ii: Call set_Nxx_dxx_invdx_params__and__xx(), which sets
    //             params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
    //             chosen Eigen-CoordSystem.
    set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, &griddata.params, griddata.xx);
    // Step 0e: Find ghostzone mappings; set up bcstruct
    // Step 0e.i: Free allocated space for xx[][] array
  }
  
  // Step 0.m: Allocate memory for non y_n_gfs. We do this here to free up
  //         memory for setting up initial data (for cases in which initial
  //         data setup is memory intensive.)
  const int Nxx_plus_2NGHOSTS0 = griddata.params.Nxx_plus_2NGHOSTS0;
  const int Nxx_plus_2NGHOSTS1 = griddata.params.Nxx_plus_2NGHOSTS1;
  const int Nxx_plus_2NGHOSTS2 = griddata.params.Nxx_plus_2NGHOSTS2;
  const int Nxx_plus_2NGHOSTS_tot = griddata.params.Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2;

  double *restrict evol_gfs    = (double *restrict)malloc(sizeof(double) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS_tot);
  double *restrict auxevol_gfs = (double *restrict)malloc(sizeof(double) * NUM_AUXEVOL_GFS * Nxx_plus_2NGHOSTS_tot);


  // Now we need to properly populate our structs for the unit test.

  reconstructed_prims_struct reconstructed_prims;
  reconstructed_prims_struct reconstructed_prims_r;
  reconstructed_prims_struct reconstructed_prims_l;

  conservative_fluxes_struct conservative_fluxes;
  conservative_sources_struct conservative_sources;

  metric_quantities_struct metric_quantities;
  metric_face_quantities_struct metric_face_quantities;
  metric_quantities_derivatives_struct metric_quantities_derivatives;

  // We begin by generating random inital data.

  for(int which_gf=0; which_gf<NUM_AUXEVOL_GFS; which_gf++) for(int ijk=0; ijk<Nxx_plus_2NGHOSTS_tot; ijk++) {
    auxevol_gfs[IDX4ptS(which_gf, ijk)] = ((double)rand() - (double)rand())/(double)(RAND_MAX);
        }
        
  for(int ijk=0; ijk<Nxx_plus_2NGHOSTS_tot; ijk++) {
      auxevol_gfs[IDX4ptS(GAMMADD00GF, ijk)] = 1.0;
      auxevol_gfs[IDX4ptS(GAMMADD01GF, ijk)] = 0.0;
      auxevol_gfs[IDX4ptS(GAMMADD02GF, ijk)] = 0.0;
      auxevol_gfs[IDX4ptS(GAMMADD11GF, ijk)] = 1.0;
      auxevol_gfs[IDX4ptS(GAMMADD12GF, ijk)] = 0.0;
      auxevol_gfs[IDX4ptS(GAMMADD22GF, ijk)] = 1.0;

      auxevol_gfs[IDX4ptS(GAMMA_FACEDD00GF, ijk)] = 1.0;
      auxevol_gfs[IDX4ptS(GAMMA_FACEDD01GF, ijk)] = 0.0;
      auxevol_gfs[IDX4ptS(GAMMA_FACEDD02GF, ijk)] = 0.0;
      auxevol_gfs[IDX4ptS(GAMMA_FACEDD11GF, ijk)] = 1.0;
      auxevol_gfs[IDX4ptS(GAMMA_FACEDD12GF, ijk)] = 0.0;
      auxevol_gfs[IDX4ptS(GAMMA_FACEDD22GF, ijk)] = 1.0;

    }

  for(int which_gf=0; which_gf<NUM_EVOL_GFS; which_gf++) for(int ijk=0; ijk<Nxx_plus_2NGHOSTS_tot; ijk++) {
    evol_gfs[IDX4ptS(which_gf, ijk)] = 0.0/0.0;
        }

  // Now we fill our structs and call the relevant functions
  for(int ijk=0; ijk<Nxx_plus_2NGHOSTS_tot; ijk++) {
  
      initialize_structs(ijk, &griddata.params, auxevol_gfs, &reconstructed_prims,
                         &reconstructed_prims_r,
                         &reconstructed_prims_l,
                         &metric_quantities,
                         &metric_face_quantities,
                         &metric_quantities_derivatives);
                         
         
     calculate_all_source_terms(&reconstructed_prims,
                                &metric_quantities,
                                &metric_quantities_derivatives,
                                &conservative_sources);
                                
    evol_gfs[IDX4ptS(STILDED0GF, ijk)]  = conservative_sources.StildeD0_src;
    evol_gfs[IDX4ptS(STILDED1GF, ijk)]  = conservative_sources.StildeD1_src;
    evol_gfs[IDX4ptS(STILDED2GF, ijk)]  = conservative_sources.StildeD2_src;
    evol_gfs[IDX4ptS(TAU_TILDEGF, ijk)] = conservative_sources.tau_tilde_src;
    evol_gfs[IDX4ptS(RHO_STARGF, ijk)]  = 0.0;

    calculate_HLLE_fluxes0(&reconstructed_prims_r, 
                           &reconstructed_prims_l,
                           &metric_face_quantities,
                           &conservative_fluxes);
                           
    evol_gfs[IDX4ptS(STILDED0GF, ijk)]  += conservative_fluxes.HLLE_flux_StildeD0;
    evol_gfs[IDX4ptS(STILDED1GF, ijk)]  += conservative_fluxes.HLLE_flux_StildeD1;
    evol_gfs[IDX4ptS(STILDED2GF, ijk)]  += conservative_fluxes.HLLE_flux_StildeD2;
    evol_gfs[IDX4ptS(TAU_TILDEGF, ijk)] += conservative_fluxes.HLLE_flux_tau_tilde;
    evol_gfs[IDX4ptS(RHO_STARGF, ijk)]  += conservative_fluxes.HLLE_flux_rho_star;

    calculate_HLLE_fluxes1(&reconstructed_prims_r, 
                           &reconstructed_prims_l,
                           &metric_face_quantities,
                           &conservative_fluxes);

    evol_gfs[IDX4ptS(STILDED0GF, ijk)]  += conservative_fluxes.HLLE_flux_StildeD0;
    evol_gfs[IDX4ptS(STILDED1GF, ijk)]  += conservative_fluxes.HLLE_flux_StildeD1;
    evol_gfs[IDX4ptS(STILDED2GF, ijk)]  += conservative_fluxes.HLLE_flux_StildeD2;
    evol_gfs[IDX4ptS(TAU_TILDEGF, ijk)] += conservative_fluxes.HLLE_flux_tau_tilde;
    evol_gfs[IDX4ptS(RHO_STARGF, ijk)]  += conservative_fluxes.HLLE_flux_rho_star;

    calculate_HLLE_fluxes2(&reconstructed_prims_r, 
                           &reconstructed_prims_l,
                           &metric_face_quantities,
                           &conservative_fluxes);

    evol_gfs[IDX4ptS(STILDED0GF, ijk)]  += conservative_fluxes.HLLE_flux_StildeD0;
    evol_gfs[IDX4ptS(STILDED1GF, ijk)]  += conservative_fluxes.HLLE_flux_StildeD1;
    evol_gfs[IDX4ptS(STILDED2GF, ijk)]  += conservative_fluxes.HLLE_flux_StildeD2;
    evol_gfs[IDX4ptS(TAU_TILDEGF, ijk)] += conservative_fluxes.HLLE_flux_tau_tilde;
    evol_gfs[IDX4ptS(RHO_STARGF, ijk)]  += conservative_fluxes.HLLE_flux_rho_star;                         
 }
 
 
 for(int ijk=0; ijk<Nxx_plus_2NGHOSTS_tot; ijk++) {
    printf("%.6e %.6e %.6e %.6e %.6e\n", evol_gfs[IDX4ptS(STILDED0GF, ijk)], 
                                              evol_gfs[IDX4ptS(STILDED1GF, ijk)], 
                                              evol_gfs[IDX4ptS(STILDED2GF, ijk)], 
                                              evol_gfs[IDX4ptS(TAU_TILDEGF, ijk)],
                                              evol_gfs[IDX4ptS(RHO_STARGF, ijk)]);
    
}

    FILE *pointsx_file = fopen("input_data1.txt", "rb");
    double *points_dest_grid_x = (double  *)malloc(sizeof(double)*14);
    fread(points_dest_grid_x, sizeof(double), 14, pointsx_file);
    fclose(pointsx_file);


  // Step 4: Free all allocated memory
  free(evol_gfs);
  free(auxevol_gfs);
  for(int i=0;i<3;i++) free(griddata.xx[i]);
  return 0;
"""

    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        rel_path_to_Cparams=os.path.join("."), enableCparameters=False)

In [10]:
# Generate & register the find_timestep() function
rfm.register_C_functions(enable_rfm_precompute=enable_rfm_precompute,
                         use_unit_wavespeed_for_find_timestep=True)
rfm.register_NRPy_basic_defines(enable_rfm_precompute=enable_rfm_precompute)


In [11]:
import outputC as outC
outC.outputC_register_C_functions_and_NRPy_basic_defines()  # #define M_PI,  etc.
# Declare paramstruct, register set_Cparameters_to_default(),
#   and output declare_Cparameters_struct.h and set_Cparameters[].h:
outC.NRPy_param_funcs_register_C_functions_and_NRPy_basic_defines(os.path.join(Ccodesrootdir))

gri.register_C_functions_and_NRPy_basic_defines(enable_griddata_struct=True,
                                                enable_bcstruct_in_griddata_struct=False,
                                                enable_rfmstruct=enable_rfm_precompute,
                                                enable_MoL_gfs_struct=False)  # #define IDX3S(),  etc.
fin.register_C_functions_and_NRPy_basic_defines(NGHOSTS_account_for_onezone_upwind=False)  # #define NGHOSTS, etc.



# Call this last: Set up NRPy_basic_defines.h and NRPy_function_prototypes.h.
outC.construct_NRPy_basic_defines_h(Ccodesrootdir, enable_SIMD=enable_SIMD)
outC.construct_NRPy_function_prototypes_h(Ccodesrootdir)

add_to_Cfunction_dict_main__IGM_Playground()


In [12]:
from mpmath import mp 

mp.dps = 40

mp.mpf(mp.sqrt(4.*mp.pi))

mpf('3.544907701811032054596334966682290365595109')

In [13]:
metric_derivs_str = ""
for var in all_free_sysmbols:
    if "_dD" in str(var):
        metric_derivs_str += "double "+str(var)+";\n"
        

module_structs_str = r"""

//#ifndef TINYDOUBLE
static const double TINYDOUBLE = 1e-100;
//#endif

//#ifndef sqrt4pi
static const double sqrt4pi = 3.544907701811032054596334966682290365L;
//#endif

static const int kronecker_delta[4][3] = { { 0,0,0 },
                                    { 1,0,0 },
                                    { 0,1,0 },
                                    { 0,0,1 } };

/*
   The struct reconstructed_prims_struct contains variables for storing the (point-wise)
   reconstructed primitive variables. The struct elements are detailed below:
   
 --rhob: the baryonic density
 
 --P: the pressure
 
 --u4U*: the contravariant fluid 4-velocity
   
 --BU*: the contravariant magnetic field
 
 --h: the enthalpy
 
 --Gamma_th: an EOS quantity
 
 --epsilon_th: an EOS quantity
 
 --dPcold_drhob: an EOS quantity
*/

typedef struct __reconstructed_prims__ {
double u4U0, u4U1, u4U2, u4U3;
double BU0, BU1, BU2;
double rhob;
double P, h, Gamma_th, epsilon_th, dPcold_drhob;
} reconstructed_prims_struct;


/*
   Parameters used throughout the module. The struct elements are detailed below:
 
 --TINYDOUBLE: tiny double to avoid divide by zero issues
 
 --sqrt4pi: \sqrt{4\pi} to 16 S.D.
*/

/*
typedef struct __rhss_params__ {
double TINYDOUBLE, sqrt4pi;
} rhss_params_struct;
*/


/*
   The struct conservative_fluxes_struct contains variables for HLLE fluxes that the user
   will finite difference later. Note that we do not store the fluxes for all three
   directions. The struct elements are detailed below:
   
 --cmin* / cmax*: the minimum and maximum characteristic speeds for all three flux directions
 
 --HLLE_flux_StildeD*: the HLLE flux for the momentum term term, all three components
 
 --HLLE_flux_rho_star: the HLLE flux for the baryonic density
   
 --HLLE_flux_tau_tilde: the HLLE flux for the energy
*/

typedef struct __conservative_fluxes__ {
double cmin_dirn0, cmin_dirn1, cmin_dirn2;
double cmax_dirn0, cmax_dirn1, cmax_dirn2;
double HLLE_flux_StildeD0, HLLE_flux_StildeD1, HLLE_flux_StildeD2;
double HLLE_flux_rho_star, HLLE_flux_tau_tilde;
} conservative_fluxes_struct;


/*
   The struct conservative_sources_struct contains stores the point-wise values 
   of the source terms for the momentum and energy equations. The struct 
   elements are detailed below:
   
 --StildeD*_src: momentum equations source terms
 
 --tau_tilde_src: tau equation source term
*/

typedef struct __conservative_sources__ {
double StildeD0_src, StildeD1_src, StildeD2_src;
double tau_tilde_src;
} conservative_sources_struct;


"""
if formalism=="BSSN":
    module_structs_str += r"""
/*
   The struct metric_quantities_struct contains variables for storing the (point-wise)
   metric variables. Note that while we use the names of rescaled BSSN variables, 
   since this module is to be used in Cartesian coordinates only, they are not actually
   rescaled. The struct elements are detailed below:
   
 --alpha: the lapse
 
 --trK: the trace of the extrinsic curvature
 
 --cf: the conformal factor, assuming cf = e^4\phi
 
 --vetU*: the rescaled shift vector
   
 --aDD**: the rescaled trace-free part of the extrinsic curvature
 
 --hDD*: the rescaled deviations from the background metric
*/

typedef struct __metric_quantities__ {
double alpha, trK, cf;
double vetU0, vetU1, vetU2;
double aDD01, aDD02, aDD12, aDD00, aDD11, aDD22;
double hDD01, hDD02, hDD12, hDD00, hDD11, hDD22;
} metric_quantities_struct;


/*
   The struct metric_face_quantities_struct contains metric quantities stored on grid faces.
*/

typedef struct __metric_face_quantities__ {
double alpha_face, cf_face;
double vet_faceU0, vet_faceU1, vet_faceU2;
double h_faceDD01, h_faceDD02, h_faceDD12, h_faceDD00, h_faceDD11, h_faceDD22;
} metric_face_quantities_struct;


"""
elif formalism=="ADM":
    module_structs_str += r"""
/*
   The struct metric_quantities_struct contains variables for storing the (point-wise)
   metric variables. The struct elements are detailed below:
   
 --alpha: the lapse
 
 --betaU*: the shift vector
   
 --gammaDD**: the physical 3-metric
 
 --KDD*: the extrinsic curvature
*/

typedef struct __metric_quantities__ {
double alpha;
double betaU0, betaU1, betaU2;
double gammaDD01, gammaDD02, gammaDD12, gammaDD00, gammaDD11, gammaDD22;
double KDD01, KDD02, KDD12, KDD00, KDD11, KDD22;
} metric_quantities_struct;


/*
   The struct metric_face_quantities_struct contains metric quantities stored on grid faces.
*/

typedef struct __metric_face_quantities__ {
double alpha_face;
double beta_faceU0, beta_faceU1, beta_faceU2;
double gamma_faceDD01, gamma_faceDD02, gamma_faceDD12, gamma_faceDD00, gamma_faceDD11, gamma_faceDD22;
} metric_face_quantities_struct;

"""
else:
    print("ERROR: invalid choice of formalism")
    sys.exit(1)

module_structs_str += r"""
/*
   The struct metric_quantities_derivatives_struct contains variables for
   precomputed derivatives of the metric quantities.
*/

typedef struct __metric_quantities_derivatives__ {
"""+metric_derivs_str+"""
} metric_quantities_derivatives_struct;

static const int cctk_lsh[3] = {1,1,1};
// Define CCTK macros
#define CCTK_REAL double
#define CCTK_INT int
typedef struct __cGH__ {
double *cctkGH;
} cGH;

// Define dz in CCTK
static CCTK_REAL cactus_dz;
#define CCTK_DELTA_SPACE(i) cactus_dz

static const CCTK_REAL GAMMA_SPEED_LIMIT = 2000.0;

// Dummy ETK function:
#define CCTK_GFINDEX3D(cctkGH,i,j,k) 0
#define CCTK_VInfo(a01,a02,a03,a04,a05,a06,a07,a08,a09,a10,a11,a12) //
#define CCTK_VWarn(b01,b02,b03,b04,b05,b06,b07,b08,b09,b10,b11,b12,b13,b14,b15,b16,b17,b18,b19,b20,b21,b22,b23,b24,b25) //

#define SQR(x) ((x) * (x))
#define ONE_OVER_SQRT_4PI 0.282094791773878143474039725780

//#define VERR_DEF_PARAMS __LINE__, __FILE__, CCTK_THORNSTRING

// The order here MATTERS, as we assume that GUPXX+1=GUPYY, etc.
static const int PHI=0,PSI=1,GXX=2,GXY=3,GXZ=4,GYY=5,GYZ=6,GZZ=7,
  LAPM1=8,SHIFTX=9,SHIFTY=10,SHIFTZ=11,GUPXX=12,GUPYY=13,GUPZZ=14,
  NUMVARS_FOR_METRIC_FACEVALS=15; //<-- Be _sure_ to set this correctly, or you'll have memory access bugs!

// These are not used for facevals in the reconstruction step, but boy are they useful anyway.
static const int GUPXY=15,GUPXZ=16,GUPYZ=17,
  NUMVARS_FOR_METRIC=18; //<-- Be _sure_ to set this correctly, or you'll have memory access bugs!

// The order here MATTERS, and must be consistent with the order in the in_prims[] array in driver_evaluate_MHD_rhs.C.
static const int RHOB=0,PRESSURE=1,VX=2,VY=3,VZ=4,
  BX_CENTER=5,BY_CENTER=6,BZ_CENTER=7,BX_STAGGER=8,BY_STAGGER=9,BZ_STAGGER=10,
  VXR=11,VYR=12,VZR=13,VXL=14,VYL=15,VZL=16,MAXNUMVARS=17;  //<-- Be _sure_ to define MAXNUMVARS appropriately!

static const int UT=0,UX=1,UY=2,UZ=3;

// The "I" suffix denotes interpolation. In other words, these
//    definitions are used for interpolation ONLY. The order here
//    matters as well!
static const int SHIFTXI=0,SHIFTYI=1,SHIFTZI=2,GUPXXI=3,GUPXYI=4,GUPXZI=5,GUPYYI=6,GUPYZI=7,GUPZZI=8,
  PSII=9,LAPM1I=10,A_XI=11,A_YI=12,A_ZI=13,LAPSE_PSI2I=14,LAPSE_OVER_PSI6I=15,MAXNUMINTERP=16;

// Again, the order here MATTERS, since we assume in the code that, e.g., smallb[0]=b^t, smallb[3]=b^z, etc.
static const int SMALLBT=0,SMALLBX=1,SMALLBY=2,SMALLBZ=3,SMALLB2=4,NUMVARS_SMALLB=5;

// Again, the order here MATTERS, since we assume in the code that, CONSERV[STILDEX+1] = \tilde{S}_y
static const int RHOSTAR=0,STILDEX=1,STILDEY=2,STILDEZ=3,TAUENERGY=4,NUM_CONSERVS=5;

static const int LAPSE=0,PSI2=1,PSI4=2,PSI6=3,PSIM4=4,LAPSEINV=5,NUMVARS_METRIC_AUX=6;
#define SET_LAPSE_PSI4(array_name,METRIC)   {                   \
      array_name[LAPSE] = METRIC[LAPM1]+1.0;                    \
      array_name[PSI2]  = exp(2.0*METRIC[PHI]);                 \
      array_name[PSI4]  = SQR(array_name[PSI2]);                \
      array_name[PSI6]  = array_name[PSI4]*array_name[PSI2];    \
      array_name[PSIM4]  = 1.0/array_name[PSI4];                \
      array_name[LAPSEINV]  = 1.0/array_name[LAPSE];            \
  }

// Keeping track of ghostzones between routines is a nightmare, so
//   we instead attach ghostzone info to each gridfunction and set
//   the ghostzone information correctly within each routine.
typedef struct __gf_and_gz_struct__ {
  CCTK_REAL *gf;
  int gz_lo[4],gz_hi[4];
} gf_and_gz_struct;

#define MAX_EOS_PARAMS 10
typedef struct __eos_struct__ {
  int neos;
  CCTK_REAL rho_tab[MAX_EOS_PARAMS],P_tab[MAX_EOS_PARAMS],eps_tab[MAX_EOS_PARAMS],k_tab[MAX_EOS_PARAMS],gamma_tab[MAX_EOS_PARAMS];
  CCTK_REAL gamma_th;
  CCTK_REAL K_poly;
} eos_struct;

typedef struct __output_stats__ {
  int font_fixed,vel_limited,failure_checker;
  long n_iter;
} output_stats;


"""

with open(os.path.join(Ccodesrootdir,"NRPy_basic_defines.h"), "a") as file:
    file.write(module_structs_str)

<a id='compileexec'></a>

# Step 6: Compile generated C codes & perform GRFFE simulations \[Back to [top](#toc)\]
$$\label{compileexec}$$

To aid in the cross-platform-compatible (with Windows, MacOS, & Linux) compilation and execution, we make use of `cmdline_helper` [(**Tutorial**)](Tutorial-cmdline_helper.ipynb).

In [14]:
import cmdline_helper as cmd
cmd.new_C_compile(Ccodesrootdir, "IGM_standalone",
                  uses_free_parameters_h=True, compiler_opt_option="debug") # fastdebug or debug also supported
os.chdir(Ccodesrootdir)
# Clean up existing output files
cmd.delete_existing_files("out*.txt")
cmd.delete_existing_files("out*.png")

(EXEC): Executing `make -j34`...
In file included from main.c:7:
compute_tau_rhs_extrinsic_curvature_terms_and_TUPmunu.C:1:13: warning: ‘compute_tau_rhs_extrinsic_curvature_terms_and_TUPmunu’ defined but not used [-Wunused-function]
    1 | static void compute_tau_rhs_extrinsic_curvature_terms_and_TUPmunu
      |             ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
In file included from main.c:6:
add_fluxes_and_source_terms_to_hydro_rhss.C:34:13: warning: ‘add_fluxes_and_source_terms_to_hydro_rhss’ defined but not used [-Wunused-function]
   34 | static void add_fluxes_and_source_terms_to_hydro_rhss(const int flux_dirn,const cGH *cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,CCTK_REAL *dX,
      |             ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
In file included from main.c:4:
reconstruct_set_of_prims_PPM.C:317:13: warning: ‘ftilde_gf_compute’ defined but not used [-Wunused-function]
  317 | static void ftilde_gf_compute(const cGH *cctkGH,const int *cctk_lsh,

In [15]:
cmd.Execute("IGM_standalone", "20 20 20 ", os.path.join("output", "out20.txt"))

# Return to root directory
os.chdir(os.path.join(".."))

(EXEC): Executing `taskset -c 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15 ./IGM_standalone 20 20 20 `...
(BENCH): Finished executing in 0.2 seconds.


<a id='plots'></a>

# Step 7: Data Visualization \[Back to [top](#toc)\]
$$\label{plots}$$

Now we plot the data and recreate figure 1 from the [GiRaFFE paper](https://arxiv.org/pdf/1704.00599.pdf). We reconstruct the electric field via

$$
E_i = -\epsilon_{ijk}v^j B^k
$$

the `calc_E` function below. We also calculate the FFE condition $B^2 - E^2$ below using the `calc_Bsquared_minus_Esquared` function.

In [16]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.facecolor'] = 'white'
# os.chdir(os.path.join(".."))
# ID options are: "AlfvenWave", "ThreeAlfvenWaves", "DegenAlfvenWave", "FastWave", "FFEBD"
ii = 0
data = ID_opts[ii]
Data_num_Fast_0 = np.loadtxt(os.path.join(Ccodesrootdir,"output","out306__"+data+"-00000000.txt"))
Data_num_Fast_A = np.loadtxt(os.path.join(Ccodesrootdir,"output","out306__"+data+"-00000400.txt"))

# BU0 = 1, AD0 = 4, VU0 = 7, cminx = 10
jj = 3

plt.plot(Data_num_Fast_0[:,0], Data_num_Fast_0[:,jj], label = 't = 0')
plt.plot(Data_num_Fast_A[:,0], Data_num_Fast_A[:,jj], label = 'num')
plt.plot(Data_num_Fast_A[:,0], Data_num_Fast_A[:,jj+15], label = 'exact')
# plt.xlim(-0.5, 1.5)
# plt.ylim(0.9, 1.36)
plt.legend()
plt.xlabel("x")
plt.ylabel("B_z")
plt.title("CurviGiRaFFE: "+data)
plt.show()

NameError: name 'ID_opts' is not defined

<a id='latex_pdf_output'></a>

# Step 8: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-Start_to_Finish-GiRaFFE_NRPy-1D_tests-staggered.pdf](Tutorial-Start_to_Finish-GiRaFFE_NRPy-1D_tests-staggered.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [ ]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-Start_to_Finish-GiRaFFE_NRPy-1D_tests-staggered",location_of_template_file=os.path.join(".."))